Trying to solve the given system of polynomial equations

ax**2+by**7.8+cz+32=0
sx**3+dy**8+ez**3+90=0

USING OGGNs:

In [1]:
import torch

#imports necessary to define a neural network 
import torch.nn as nn
import torch.nn.functional as F

#ensure you are using GPU.
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  

device = torch.device(dev)
print(device)

dtype = torch.cuda.FloatTensor

cuda:0


In [2]:
import numpy as np

In [3]:
a0=9.0
a1=8.97
a2=0.876

def output1(x0,x1,x2):
    
    out=a0*x0**(2.0)
    out=out+a1*x1**(7.8)
    out=out+a2*x2**(1.0)
    out=out-32.0
    
    return out

In [4]:
a00=12.0
a10=9.97
a20=10.876

def output2(x0,x1,x2):
    
    out=a00*x0**(3.0)
    out=out+a10*x1**(8.0)
    out=out+a20*x2**(3.0)
    out=out-43.0
    
    return out

In [5]:
def finalloss(x):
    
    f=torch.empty([len(x),1]).type(dtype)
    
    for i in range(len(x)):
        
        x0=x[i][0]
        x1=x[i][1]
        x2=x[i][2]

        f[i][0]=(output1(x0,x1,x2)**2+output2(x0,x1,x2)**2)**(0.5)
    
    return f

In [6]:
finalloss([[0.431, 0.01026, 0.0000]])

tensor([[51.8372]], device='cuda:0')

In [7]:
class Generator(nn.Module):
    def __init__(self):
    
        super(Generator, self).__init__()


        self.fc1 = nn.Linear(40,800)
        self.fc2 = nn.Linear(800,160)
        self.fc4=nn.Linear(160,16)
        self.fc3=nn.Linear(16,3)
        '''
        
        self.fc3 = nn.Linear(6400, 12800)

        self.fc5=nn.Linear(12800, 12800)
        self.fc6=nn.Linear(12800, 1280)
        self.fc4 = nn.Linear(1280, 784)


        self.dropout = nn.Dropout(0.3)
        '''
    def forward(self, x):
        
        x = F.relu(self.fc1(x)) # (input, negative_slope=0.2)
        
        x = F.relu(self.fc2(x))
        
        x = F.relu(self.fc4(x))
        
        x = F.relu(self.fc3(x))
        
        '''
        out =F.relu(self.fc5(x))
        out =F.relu(self.fc6(out))
        out =F.relu(self.fc4(out))
        out[out>255]=out[out>255]/100
        out[out<55]=out[out<55]*1000
        '''
        #x[x==0]=x[x==0]+10
        return x

In [8]:
def customloss(inp,bsize):
    
    cusout=finalloss(inp.reshape([bsize,3])).type(dtype)
    
    cusout=cusout.reshape([bsize,1]).type(dtype)
    cusideal=200*torch.zeros([bsize,1]).type(dtype)
    
    cuscriterion= nn.MSELoss()
    
    loss= cuscriterion(cusout,cusideal).type(dtype)
    
    return loss

In [9]:
xgarb=torch.rand([100,40]).type(dtype)

In [10]:
import torch.optim as optim

learning_rate=0.001
momentum=0.05


gennetwork = Generator().cuda()
genoptimizer = optim.Adagrad(gennetwork.parameters(), lr=learning_rate)
print(gennetwork)

Generator(
  (fc1): Linear(in_features=40, out_features=800, bias=True)
  (fc2): Linear(in_features=800, out_features=160, bias=True)
  (fc4): Linear(in_features=160, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=3, bias=True)
)


In [11]:
gennetwork(xgarb[0:2].reshape(2,40))

tensor([[0.0732, 0.0000, 0.1009],
        [0.0664, 0.0000, 0.0862]], device='cuda:0', grad_fn=<ReluBackward0>)

In [12]:
batchsize=2

batches=len(xgarb)/batchsize

epochs=4000

for i in range(epochs):
    
    epochloss=0
        
    for j in range(int(batches)):
        
        
        #print(Ytensor)
        out=gennetwork(xgarb[j*batchsize:(j+1)*batchsize].reshape([batchsize,40])).type(dtype)

        #compute loss
        loss = customloss(out,batchsize).type(dtype)


        #backprop loss i.e. find dloss/dparam for each parameter and store.
        loss.backward(retain_graph=True)
        
        #clip gradients
        torch.nn.utils.clip_grad_norm_(gennetwork.parameters(), 1.0)
        genoptimizer.step()
        
        epochloss=epochloss+loss
        
    print(epochloss,i)

tensor(48636.4805, device='cuda:0', grad_fn=<AddBackward0>) 0
tensor(9420.4092, device='cuda:0', grad_fn=<AddBackward0>) 1
tensor(8764.6738, device='cuda:0', grad_fn=<AddBackward0>) 2
tensor(8110.8154, device='cuda:0', grad_fn=<AddBackward0>) 3
tensor(7687.9160, device='cuda:0', grad_fn=<AddBackward0>) 4
tensor(7441.5562, device='cuda:0', grad_fn=<AddBackward0>) 5
tensor(7348.7705, device='cuda:0', grad_fn=<AddBackward0>) 6
tensor(7016.1636, device='cuda:0', grad_fn=<AddBackward0>) 7
tensor(6473.7212, device='cuda:0', grad_fn=<AddBackward0>) 8
tensor(5734.6353, device='cuda:0', grad_fn=<AddBackward0>) 9
tensor(5312.7358, device='cuda:0', grad_fn=<AddBackward0>) 10
tensor(5135.5552, device='cuda:0', grad_fn=<AddBackward0>) 11
tensor(5228.8516, device='cuda:0', grad_fn=<AddBackward0>) 12
tensor(5130.0864, device='cuda:0', grad_fn=<AddBackward0>) 13
tensor(5461.2510, device='cuda:0', grad_fn=<AddBackward0>) 14
tensor(5029.9746, device='cuda:0', grad_fn=<AddBackward0>) 15
tensor(4563.3745,

tensor(285.8039, device='cuda:0', grad_fn=<AddBackward0>) 133
tensor(278.2721, device='cuda:0', grad_fn=<AddBackward0>) 134
tensor(261.8027, device='cuda:0', grad_fn=<AddBackward0>) 135
tensor(278.9975, device='cuda:0', grad_fn=<AddBackward0>) 136
tensor(285.6764, device='cuda:0', grad_fn=<AddBackward0>) 137
tensor(285.2721, device='cuda:0', grad_fn=<AddBackward0>) 138
tensor(249.3639, device='cuda:0', grad_fn=<AddBackward0>) 139
tensor(271.5663, device='cuda:0', grad_fn=<AddBackward0>) 140
tensor(233.7097, device='cuda:0', grad_fn=<AddBackward0>) 141
tensor(250.4607, device='cuda:0', grad_fn=<AddBackward0>) 142
tensor(255.7912, device='cuda:0', grad_fn=<AddBackward0>) 143
tensor(263.0157, device='cuda:0', grad_fn=<AddBackward0>) 144
tensor(223.8222, device='cuda:0', grad_fn=<AddBackward0>) 145
tensor(226.3119, device='cuda:0', grad_fn=<AddBackward0>) 146
tensor(212.0815, device='cuda:0', grad_fn=<AddBackward0>) 147
tensor(204.0398, device='cuda:0', grad_fn=<AddBackward0>) 148
tensor(2

tensor(14.0478, device='cuda:0', grad_fn=<AddBackward0>) 267
tensor(15.1408, device='cuda:0', grad_fn=<AddBackward0>) 268
tensor(18.4997, device='cuda:0', grad_fn=<AddBackward0>) 269
tensor(13.6717, device='cuda:0', grad_fn=<AddBackward0>) 270
tensor(15.6155, device='cuda:0', grad_fn=<AddBackward0>) 271
tensor(13.8246, device='cuda:0', grad_fn=<AddBackward0>) 272
tensor(13.9606, device='cuda:0', grad_fn=<AddBackward0>) 273
tensor(16.8276, device='cuda:0', grad_fn=<AddBackward0>) 274
tensor(13.4310, device='cuda:0', grad_fn=<AddBackward0>) 275
tensor(13.9703, device='cuda:0', grad_fn=<AddBackward0>) 276
tensor(10.1583, device='cuda:0', grad_fn=<AddBackward0>) 277
tensor(12.6887, device='cuda:0', grad_fn=<AddBackward0>) 278
tensor(13.6034, device='cuda:0', grad_fn=<AddBackward0>) 279
tensor(13.6287, device='cuda:0', grad_fn=<AddBackward0>) 280
tensor(13.8124, device='cuda:0', grad_fn=<AddBackward0>) 281
tensor(12.8737, device='cuda:0', grad_fn=<AddBackward0>) 282
tensor(13.5892, device='

tensor(7.9085, device='cuda:0', grad_fn=<AddBackward0>) 403
tensor(7.8371, device='cuda:0', grad_fn=<AddBackward0>) 404
tensor(7.1937, device='cuda:0', grad_fn=<AddBackward0>) 405
tensor(6.9111, device='cuda:0', grad_fn=<AddBackward0>) 406
tensor(8.0887, device='cuda:0', grad_fn=<AddBackward0>) 407
tensor(8.4697, device='cuda:0', grad_fn=<AddBackward0>) 408
tensor(8.1256, device='cuda:0', grad_fn=<AddBackward0>) 409
tensor(7.9666, device='cuda:0', grad_fn=<AddBackward0>) 410
tensor(8.6892, device='cuda:0', grad_fn=<AddBackward0>) 411
tensor(6.3133, device='cuda:0', grad_fn=<AddBackward0>) 412
tensor(5.8586, device='cuda:0', grad_fn=<AddBackward0>) 413
tensor(8.3255, device='cuda:0', grad_fn=<AddBackward0>) 414
tensor(8.6974, device='cuda:0', grad_fn=<AddBackward0>) 415
tensor(7.3795, device='cuda:0', grad_fn=<AddBackward0>) 416
tensor(9.6149, device='cuda:0', grad_fn=<AddBackward0>) 417
tensor(8.3591, device='cuda:0', grad_fn=<AddBackward0>) 418
tensor(6.6967, device='cuda:0', grad_fn=

tensor(4.9676, device='cuda:0', grad_fn=<AddBackward0>) 540
tensor(5.9556, device='cuda:0', grad_fn=<AddBackward0>) 541
tensor(5.3407, device='cuda:0', grad_fn=<AddBackward0>) 542
tensor(5.9615, device='cuda:0', grad_fn=<AddBackward0>) 543
tensor(5.6196, device='cuda:0', grad_fn=<AddBackward0>) 544
tensor(5.0362, device='cuda:0', grad_fn=<AddBackward0>) 545
tensor(5.0628, device='cuda:0', grad_fn=<AddBackward0>) 546
tensor(5.8012, device='cuda:0', grad_fn=<AddBackward0>) 547
tensor(5.0803, device='cuda:0', grad_fn=<AddBackward0>) 548
tensor(4.8287, device='cuda:0', grad_fn=<AddBackward0>) 549
tensor(5.2311, device='cuda:0', grad_fn=<AddBackward0>) 550
tensor(6.1412, device='cuda:0', grad_fn=<AddBackward0>) 551
tensor(6.1382, device='cuda:0', grad_fn=<AddBackward0>) 552
tensor(4.4755, device='cuda:0', grad_fn=<AddBackward0>) 553
tensor(6.0829, device='cuda:0', grad_fn=<AddBackward0>) 554
tensor(7.3076, device='cuda:0', grad_fn=<AddBackward0>) 555
tensor(6.3105, device='cuda:0', grad_fn=

...

tensor(0.7824, device='cuda:0', grad_fn=<AddBackward0>) 3383
tensor(0.8204, device='cuda:0', grad_fn=<AddBackward0>) 3384
tensor(1.0250, device='cuda:0', grad_fn=<AddBackward0>) 3385
tensor(0.9387, device='cuda:0', grad_fn=<AddBackward0>) 3386
tensor(0.5641, device='cuda:0', grad_fn=<AddBackward0>) 3387
tensor(0.9289, device='cuda:0', grad_fn=<AddBackward0>) 3388
tensor(0.8532, device='cuda:0', grad_fn=<AddBackward0>) 3389
tensor(0.6822, device='cuda:0', grad_fn=<AddBackward0>) 3390
tensor(0.5969, device='cuda:0', grad_fn=<AddBackward0>) 3391
tensor(1.0399, device='cuda:0', grad_fn=<AddBackward0>) 3392
tensor(0.8460, device='cuda:0', grad_fn=<AddBackward0>) 3393
tensor(0.9608, device='cuda:0', grad_fn=<AddBackward0>) 3394
tensor(0.9618, device='cuda:0', grad_fn=<AddBackward0>) 3395
tensor(1.0417, device='cuda:0', grad_fn=<AddBackward0>) 3396
tensor(0.7211, device='cuda:0', grad_fn=<AddBackward0>) 3397
tensor(0.9285, device='cuda:0', grad_fn=<AddBackward0>) 3398
tensor(0.8115, device='c

tensor(0.8974, device='cuda:0', grad_fn=<AddBackward0>) 3518
tensor(0.7365, device='cuda:0', grad_fn=<AddBackward0>) 3519
tensor(0.7721, device='cuda:0', grad_fn=<AddBackward0>) 3520
tensor(0.9555, device='cuda:0', grad_fn=<AddBackward0>) 3521
tensor(1.0126, device='cuda:0', grad_fn=<AddBackward0>) 3522
tensor(1.1377, device='cuda:0', grad_fn=<AddBackward0>) 3523
tensor(0.9948, device='cuda:0', grad_fn=<AddBackward0>) 3524
tensor(0.8096, device='cuda:0', grad_fn=<AddBackward0>) 3525
tensor(0.8106, device='cuda:0', grad_fn=<AddBackward0>) 3526
tensor(0.9394, device='cuda:0', grad_fn=<AddBackward0>) 3527
tensor(0.8304, device='cuda:0', grad_fn=<AddBackward0>) 3528
tensor(0.9681, device='cuda:0', grad_fn=<AddBackward0>) 3529
tensor(0.8484, device='cuda:0', grad_fn=<AddBackward0>) 3530
tensor(0.7431, device='cuda:0', grad_fn=<AddBackward0>) 3531
tensor(0.7605, device='cuda:0', grad_fn=<AddBackward0>) 3532
tensor(0.8037, device='cuda:0', grad_fn=<AddBackward0>) 3533
tensor(0.6139, device='c

tensor(0.7043, device='cuda:0', grad_fn=<AddBackward0>) 3653
tensor(0.6845, device='cuda:0', grad_fn=<AddBackward0>) 3654
tensor(0.8758, device='cuda:0', grad_fn=<AddBackward0>) 3655
tensor(0.8066, device='cuda:0', grad_fn=<AddBackward0>) 3656
tensor(0.8400, device='cuda:0', grad_fn=<AddBackward0>) 3657
tensor(0.9181, device='cuda:0', grad_fn=<AddBackward0>) 3658
tensor(0.6895, device='cuda:0', grad_fn=<AddBackward0>) 3659
tensor(0.6442, device='cuda:0', grad_fn=<AddBackward0>) 3660
tensor(0.8553, device='cuda:0', grad_fn=<AddBackward0>) 3661
tensor(0.9006, device='cuda:0', grad_fn=<AddBackward0>) 3662
tensor(0.8115, device='cuda:0', grad_fn=<AddBackward0>) 3663
tensor(0.7907, device='cuda:0', grad_fn=<AddBackward0>) 3664
tensor(0.8255, device='cuda:0', grad_fn=<AddBackward0>) 3665
tensor(0.7529, device='cuda:0', grad_fn=<AddBackward0>) 3666
tensor(0.7847, device='cuda:0', grad_fn=<AddBackward0>) 3667
tensor(0.9002, device='cuda:0', grad_fn=<AddBackward0>) 3668
tensor(0.8101, device='c

tensor(0.9162, device='cuda:0', grad_fn=<AddBackward0>) 3788
tensor(0.6645, device='cuda:0', grad_fn=<AddBackward0>) 3789
tensor(0.9048, device='cuda:0', grad_fn=<AddBackward0>) 3790
tensor(0.7052, device='cuda:0', grad_fn=<AddBackward0>) 3791
tensor(0.6860, device='cuda:0', grad_fn=<AddBackward0>) 3792
tensor(0.7086, device='cuda:0', grad_fn=<AddBackward0>) 3793
tensor(0.6306, device='cuda:0', grad_fn=<AddBackward0>) 3794
tensor(0.8396, device='cuda:0', grad_fn=<AddBackward0>) 3795
tensor(0.7390, device='cuda:0', grad_fn=<AddBackward0>) 3796
tensor(0.8078, device='cuda:0', grad_fn=<AddBackward0>) 3797
tensor(0.7191, device='cuda:0', grad_fn=<AddBackward0>) 3798
tensor(0.7318, device='cuda:0', grad_fn=<AddBackward0>) 3799
tensor(0.7184, device='cuda:0', grad_fn=<AddBackward0>) 3800
tensor(0.8114, device='cuda:0', grad_fn=<AddBackward0>) 3801
tensor(0.5918, device='cuda:0', grad_fn=<AddBackward0>) 3802
tensor(0.7606, device='cuda:0', grad_fn=<AddBackward0>) 3803
tensor(0.7480, device='c

tensor(0.5702, device='cuda:0', grad_fn=<AddBackward0>) 3923
tensor(0.7851, device='cuda:0', grad_fn=<AddBackward0>) 3924
tensor(0.6946, device='cuda:0', grad_fn=<AddBackward0>) 3925
tensor(0.5241, device='cuda:0', grad_fn=<AddBackward0>) 3926
tensor(0.7672, device='cuda:0', grad_fn=<AddBackward0>) 3927
tensor(0.6439, device='cuda:0', grad_fn=<AddBackward0>) 3928
tensor(0.6123, device='cuda:0', grad_fn=<AddBackward0>) 3929
tensor(0.7378, device='cuda:0', grad_fn=<AddBackward0>) 3930
tensor(0.8190, device='cuda:0', grad_fn=<AddBackward0>) 3931
tensor(0.7936, device='cuda:0', grad_fn=<AddBackward0>) 3932
tensor(0.5727, device='cuda:0', grad_fn=<AddBackward0>) 3933
tensor(0.5468, device='cuda:0', grad_fn=<AddBackward0>) 3934
tensor(0.7598, device='cuda:0', grad_fn=<AddBackward0>) 3935
tensor(0.5884, device='cuda:0', grad_fn=<AddBackward0>) 3936
tensor(0.5350, device='cuda:0', grad_fn=<AddBackward0>) 3937
tensor(0.7893, device='cuda:0', grad_fn=<AddBackward0>) 3938
tensor(0.6774, device='c

In [13]:
out

tensor([[1.2279, 1.0952, 0.2624],
        [1.2255, 1.0951, 0.2779]], device='cuda:0', grad_fn=<ReluBackward0>)

In [14]:
print(output1(out[0][0],out[0][1],out[0][2]))

tensor(0.0376, device='cuda:0', grad_fn=<SubBackward0>)


In [15]:
print(output2(out[0][0],out[0][1],out[0][2]))

tensor(0.0557, device='cuda:0', grad_fn=<SubBackward0>)


In [16]:
print(output1(out[0][0],out[0][1],out[0][2])*output2(out[0][0],out[0][1],out[0][2]))

tensor(0.0021, device='cuda:0', grad_fn=<MulBackward0>)


In [17]:
print(finalloss(out))

tensor([[0.0672],
        [0.0579]], device='cuda:0', grad_fn=<CopySlices>)


In [18]:
int(0.7)

0